# Numerical linear algebra

The following operations are supported **in-place** and **allocation-free**
+ Matrix inversion in $n^3$ flops. Recall that inversion by Cholesky costs $(5/3)n^3$ flops and needs to allocate a matrix of same size.
+ Computation of determinant
+ Check (strict) positive definiteness

In [1]:
import sweepystats as sw
import numpy as np

def random_symmetric_matrix(n, eigmin=float('-inf')):
    """
    Simulates an column-major double-precision
    n*n symmetric matrix with minimum eigenvalue `eigmin`
    """
    A = np.random.rand(n, n)
    A = 0.5 * (A + A.T)
    # force eigenvalues to be >= eigmin
    if eigmin > float('-inf'):
        eval, evec = np.linalg.eig(A)
        eval[np.where(eval < 0)[0]] = eigmin
        A = evec * np.diag(eval) * evec.T
    # convert to column major
    return np.array(A, order='F')

Matrix inverse:

In [2]:
# A.sweep() converts A.A into -inv(A.A)
data = random_symmetric_matrix(100)
Ainv = np.linalg.inv(data)
A = sw.SweepMatrix(data)
A.sweep()
np.allclose(A.A, -Ainv) # check answer is correct

100%|████████████████████████████████████████████| 100/100 [00:00<00:00, 12265.48it/s]


True

Determinants:

In [3]:
# Determinants is computed by A.det(). Original matrix is untouched by default. 
data = random_symmetric_matrix(100)
Adet = np.linalg.det(data)
A = sw.SweepMatrix(data)
np.allclose(A.det(restore=True), Adet) # check answer is correct

100%|███████████████████████████████████████████| 100/100 [00:00<00:00, 131277.12it/s]


True

Check positive definiteness:

In [4]:
A = sw.SweepMatrix(random_symmetric_matrix(100, eigmin=0.0001)) # this is PD
A.isposdef(verbose=False)

True

In [5]:
A = sw.SweepMatrix(random_symmetric_matrix(100, eigmin=-10)) # this has negative eigenvalues, should return false
A.isposdef(verbose=False)

False

In [6]:
A = sw.SweepMatrix(random_symmetric_matrix(100, eigmin=0.0)) # this is PSD, should return false
A.isposdef(verbose=False)

False